PS-6 Neha Palsokar

collaborators: Sahil Agarwal, Bhumika khandelwal

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score
import math
from statistics import mode
import seaborn as sns
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
#1.1 Load data
titanic = pd.read_csv('titanic (1).csv.bz2')
titanic.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [3]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 14 columns):
pclass       1309 non-null int64
survived     1309 non-null int64
name         1309 non-null object
sex          1309 non-null object
age          1046 non-null float64
sibsp        1309 non-null int64
parch        1309 non-null int64
ticket       1309 non-null object
fare         1308 non-null float64
cabin        295 non-null object
embarked     1307 non-null object
boat         486 non-null object
body         121 non-null float64
home.dest    745 non-null object
dtypes: float64(3), int64(4), object(7)
memory usage: 143.2+ KB


In [4]:
#Missing value
titanic.isnull().sum()

pclass          0
survived        0
name            0
sex             0
age           263
sibsp           0
parch           0
ticket          0
fare            1
cabin        1014
embarked        2
boat          823
body         1188
home.dest     564
dtype: int64

In [5]:
summary = (titanic.groupby('survived').mean()).T
summary.columns = ['Drowned','Survived']
summary

,Drowned,Survived
pclass,2.500618,1.962000
age,30.545369,28.918228
sibsp,0.521632,0.462000
parch,0.328801,0.476000
fare,23.353831,49.361184
body,160.809917,NaN


In [6]:
# Filling Nan values for Embarked

mode_embarked = mode(titanic['embarked'])
titanic['embarked'] = titanic['embarked'].fillna(mode_embarked)
encoder = LabelEncoder()
titanic['embarked'] = encoder.fit_transform(titanic['embarked'])

titanic['sex'] = [0 if i=='male' else 1 for i in titanic.sex]

In [7]:
cols=['pclass','age','sibsp','parch','fare',"body","sex",'embarked']
survived=[]
drowned=[]
missing = []
for i in cols:
    survived.append(titanic[titanic.survived==1][i].mean())
    drowned.append(titanic[titanic.survived==0][i].mean())
    missing.append(titanic[i].isnull().sum())


In [8]:
tot = pd.DataFrame()
tot['cols']=cols
tot['survived']=survived
tot['drowned']=drowned
tot['missing_values']=missing
tot

,cols,survived,drowned,missing_values
0,pclass,1.962000,2.500618,0
1,age,28.918228,30.545369,263
2,sibsp,0.462000,0.521632,0
3,parch,0.476000,0.328801,0
4,fare,49.361184,23.353831,1
5,body,NaN,160.809917,1188
6,sex,0.678000,0.156984,0
7,embarked,1.312000,1.605686,0


In [9]:
# Filling missing values for age with the mean
titanic['age'] = titanic.age.fillna(titanic.age.mean())


Implement the decision tree

In [10]:

# Defining X and Y
X = titanic[['pclass', 'age', 'sibsp', 'parch', 'fare', 'sex','embarked']]
Y = titanic['survived']
# Splitting the data in train and test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=42)
training_set = pd.concat([X_train,y_train],axis=1)
test_set = pd.concat([X_test,y_test],axis=1)

In [11]:
def calc_entropy(length_survived,length_drowned):

    #Calulate total length
    total = length_survived + length_drowned
    
    #initialize entropy to Zero
    entropy = 0
    
    #Calculate probability
    p = length_survived / total
    if (p != 0):
        entropy += p * math.log(p, 2)
    p = (length_drowned)/total
    if (p != 0):
        entropy += p * math.log(p, 2)

    entropy = -entropy
    return entropy


In [12]:
def information_gain(data,feature):
  
    gain = 0 # intialize gain
    shape = data.shape[0] # total
    
    for i in data[feature].unique():
        
        feature_data = data[data[feature] == i]
        # Calculate weight
        weight = len(feature_data)
        
        a = len(feature_data[feature_data['survived'] == 1])
        b = len(feature_data[feature_data['survived'] == 0])
        
        #calculate information gain
        gain = gain + ((calc_entropy(a, b))*weight/shape)
    
    return gain

In [13]:
def entropy_age(dataset):    
    survived = dataset[dataset.survived==1].shape[0]
    total = dataset.shape[0];

    entropy = 0
    p = survived / total
    if (p != 0):
        entropy += p * math.log(p, 2)
    p = (total - survived)/total
    if (p != 0):
        entropy += p * math.log(p, 2)

    entropy = -entropy
    return entropy

age = np.unique(X_train['age'])
age = age[~np.isnan(age)]


# Finding the entropy for the first value of age vector

a = training_set[training_set.age<=age[0]]
b = training_set[training_set.age>age[0]]
entropy = ((a.shape[0]/(a.shape[0]+b.shape[0]))*entropy_age(a))  +  ((b.shape[0]/(a.shape[0]+b.shape[0]))*entropy_age(b))
entropy 

0.945246650784105

In [14]:
entropy  = pd.DataFrame(columns = ['Value', 'H(S)'])

#Iterating over each value of age
for i in age:
    # Splitting age according to threshold in this iteration
    training_set['age*'] = np.where(training_set['age'] > i, 1,0)
    
    # Finding entropy of age column
    gain = information_gain(training_set, 'age*')
    
    idx = len(entropy) + 1
    
    entropy.loc[idx] = [i, gain]

entropy[entropy['Value']==8]

,Value,H(S)
14,8.0,0.932815


In [15]:
#best value to split age is 8

entropy  = pd.DataFrame(columns = ['Value', 'H(S)'])

#Iterating over each value of age
for i in training_set.pclass.unique():
    
    # Splitting pclass according to threshold in this iteration
    training_set['pclass*'] = np.where(training_set['pclass'] > i, 1,0)
    
    # Finding information gain
    gain = information_gain(training_set, 'pclass*')
    
    idx = len(entropy) + 1
    
    entropy.loc[idx] = [i, gain]

entropy.sort_values(by='H(S)')

,Value,H(S)
2,2.0,0.894528
3,1.0,0.896097
1,3.0,0.946637


In [16]:
training_set = training_set.drop(['age*','pclass*'],axis=1)
training_set.columns

Index(['pclass', 'age', 'sibsp', 'parch', 'fare', 'sex', 'embarked',
       'survived'],
      dtype='object')

In [17]:
# Defining the best split values for each features beased in the lowest entropy

cols = ['pclass', 'sex','age','sibsp','embarked']

result = pd.DataFrame(columns = ['feature', 'entropy', 'value'])

for i in cols:
        
        for elem in training_set[i].unique():
            
            training_set['class'] = np.where(training_set[i] > elem, 1,0)
            
            en = information_gain(training_set, 'class')
            idx = len(result) + 1
            result.loc[idx] = [i,en,elem]


result.sort_values(by='entropy').head(21)

,feature,entropy,value
4,sex,0.736954,0
2,pclass,0.894528,2
3,pclass,0.896097,1
109,embarked,0.923385,0
110,embarked,0.930889,1
52,age,0.932815,8
12,age,0.933450,9
31,age,0.933602,5
90,age,0.933683,7
45,age,0.934535,6


In [18]:
def split(data, features):
    
    result  = pd.DataFrame(columns = ['feature', 'entropy', 'value'])
    for i in features:
        
        for elem in data[i].unique():
            
            data['class'] = np.where(data[i] > elem, 1,0)
            
            en = information_gain(data, 'class')
            idx = len(result) + 1
            
            result.loc[idx] = [i,en,elem]
    return(result[result['entropy'] == result['entropy'].min()])

In [19]:
split(training_set,['pclass', 'age', 'sibsp', 'parch', 'fare', 'sex', 'embarked'])

,feature,entropy,value
377,sex,0.736954,0


In [20]:
def percentage_survived(data):
   
    if(len(data['survived'].unique()) > 0):
        a = len(data[data['survived'] == 1])/len(data)*100
        b = len(data[data['survived'] == 0])/len(data)*100
    
    else:   
        if(data['survived'].unique() == 0):
            a = 100
            b = 0
        else:
            a = 100
            b = 0

    return(a, b)

In [21]:
training_set = training_set.drop('class',axis=1)
training_set = training_set.dropna()
    

In [22]:
result_tree = pd.DataFrame(columns = ['node', 'observation', 'Survived', 'Drowned', 'feature', 'value'])

In [23]:
#making a decision tree:
def decision_tree(data, path):
   
    cols = data.columns.drop(['survived'])
    
    #Split on the best boundary for the lowest entropy feature
    a = split(data, cols)
    
    if(np.shape(a)[0]>1):
        return(1)
    
    #Calculating the percentage of people survived and drowned at every step
    survived, drowned = percentage_survived(data)
    
    feature = a['feature'].item()
    best_value = a['value'].item()
    
    # Defining the left and right branch
    data_left = data[data[feature] > best_value]
    data_right = data.drop(index = data_left.index)
    
    #Populating the final dataset
    idx = len(result_tree) + 1
    result_tree.loc[idx] = [path, len(data),survived, drowned, feature, best_value]
    
    #Setting the stoping criterion
    if(survived == 0 or drowned == 0):
        idx = len(result_tree) + 1
        pathleaf = path + 'leaf'
        result_tree.loc[idx] = [pathleaf, len(data),survived, drowned, feature, best_value]
        return(1)
    else:
        pathl = path + 'L'
        pathr = path + 'R'
        decision_tree(data_left, pathl)
        decision_tree(data_right, pathr) 

In [24]:
decision_tree(training_set, 'r')

C:\Users\Neha\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [25]:
result_tree[result_tree['node']=='r']

,node,observation,Survived,Drowned,feature,value
1,r,1046,36.520076,63.479924,sex,0


In [26]:
result_tree[result_tree['node']=='rL']

,node,observation,Survived,Drowned,feature,value
2,rL,370,71.351351,28.648649,pclass,2


In [27]:
result_tree[result_tree['node']=='rR']

,node,observation,Survived,Drowned,feature,value
37,rR,676,17.455621,82.544379,age,9


In [28]:
result_tree[result_tree['node']=='rLL']

,node,observation,Survived,Drowned,feature,value
3,rLL,175,46.857143,53.142857,fare,23.25


In [29]:
result_tree[result_tree['node']=='rLR']

,node,observation,Survived,Drowned,feature,value
23,rLR,195,93.333333,6.666667,fare,31.6792


In [30]:
result_tree[result_tree['node']=='rRL']

,node,observation,Survived,Drowned,feature,value
38,rRL,641,15.132605,84.867395,pclass,1


In [31]:
result_tree[result_tree['node']=='rRR']

,node,observation,Survived,Drowned,feature,value
71,rRR,35,60.0,40.0,sibsp,2


In [32]:
training_set = training_set.drop('class',axis=1)

In [33]:
# Prediction function for the decision tree
path = 'r'
def prediction(row, path):
   
    rasta = result_tree[result_tree['node'] == path]

    if(len(rasta.index) == 0):
        path = path[:-1]
        rasta = result_tree[result_tree['node'] == path]
        x = np.where(rasta['Survived'].item() > rasta['Drowned'].item(), 1, 0)
        return(x.item())
    
    else:
        if(row[rasta['feature'].item()].item() > rasta['value'].item()):
            path = path + 'L'
            return(prediction(row, path))
        
        else:
            path = path + 'R'
            return(prediction(row, path))

In [34]:
# Checking on the test set

results = []

#Iterating over the test set
for elem in test_set.iterrows():
    temp = pd.DataFrame(list(elem)[1:], columns = ['pclass', 'age', 'sibsp', 'parch', 'fare', 'sex', 'embarked'])
    path = 'r'
    t = prediction(temp, 'r')
    results.append(t)

print(results)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1]


In [35]:
test_set['p'] = results

In [36]:
print("accuracy score:")
print((accuracy_score(test_set['survived'],test_set['p']))*100)
print("precision score:")
print(precision_score(test_set['survived'],test_set['p']))
print("recall score:")
print(recall_score(test_set['survived'],test_set['p']))

accuracy score:
75.19083969465649
precision score:
0.8271604938271605
recall score:
0.5677966101694916


Bagging

In [38]:

bagging = pd.DataFrame(columns = ['bag1', 'bag2','bag3','bag4','bag5'])


for elem in bagging.columns:
    
    #Defining a tree to store decision tree results for each individual bag
    result_tree = pd.DataFrame(columns = ['node', 'observation', 'Survived', 'Drowned', 'feature', 'value'])
     
    bag_train = training_set.sample(n=300, replace = True)
    decision_tree(bag_train, 'r')
    
    result = []
    
    d_final_test = test_set[['pclass', 'age', 'sibsp', 'parch', 'fare', 'sex', 'embarked']].dropna()
    
    #Making predictions on the test set
    for e in d_final_test.iterrows():
        temp = pd.DataFrame(list(e)[1:], columns = ['pclass', 'age', 'sibsp', 'parch', 'fare', 'sex', 'embarked'])
        t = prediction(temp,'r')
        result.append(t)
    #Populating the bagging dataset with predictions for each bag    
    bagging[elem] = result

C:\Users\Neha\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [39]:
# Predictions for each bag
bagging.head(15)

,bag1,bag2,bag3,bag4,bag5
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,1,0,0,0,0
5,0,0,0,0,0
6,0,0,0,0,0
7,0,0,0,0,0
8,0,0,0,0,0
9,1,1,1,1,1


In [40]:
bagging['majority']=''
for i in range(0,len(bagging)):
    bagging['majority'].iloc[i] = mode(list(bagging.iloc[i])[0:5])

C:\Users\Neha\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [41]:
bagging.tail(10)

,bag1,bag2,bag3,bag4,bag5,majority
252,0,1,0,1,0,0
253,1,1,1,1,1,1
254,0,0,0,0,0,0
255,0,0,0,0,0,0
256,1,0,1,0,0,0
257,1,1,1,1,1,1
258,0,0,1,0,0,0
259,0,1,0,1,0,0
260,0,0,1,0,0,0
261,1,1,1,1,1,1


In [42]:
print("accuracy score:")
(accuracy_score(test_set['survived'],bagging['majority']))*100
print("precision score:")
precision_score(test_set['survived'],bagging['majority'])
print("recall score:")
recall_score(test_set['survived'],bagging['majority'])

accuracy score:
precision score:
recall score:


0.5084745762711864

In [43]:
bags = [5,9]
acc=[]
p=[]
r=[]
for x in bags:
    print('Bags:',x+1)
    cols = []
    for i in (range(0,x)):
        a = 'bag'+str(i)
        cols.append(a)

    bagging = pd.DataFrame(columns = cols)

    for elem in bagging.columns:
        
        #Defining a tree to store decision tree results for each individual bag
        result_tree = pd.DataFrame(columns = ['node', 'observation', 'Survived', 'Drowned', 'feature', 'value'])

        #Drawing N samples with replacement from N observations
        bag_train = training_set.sample(n=len(training_set), replace = True)

        #Running decision tree to populate result_tree dataframe
        decision_tree(bag_train, 'r')
        
        print(result_tree.head(3))
        
        result = []

        d_final_test = test_set[['pclass', 'age', 'sibsp', 'parch', 'fare', 'sex', 'embarked']].dropna()

        #Making predictions on the test set
        for e in d_final_test.iterrows():
            temp = pd.DataFrame(list(e)[1:], columns = ['pclass', 'age', 'sibsp', 'parch', 'fare', 'sex', 'embarked'])
            t = prediction(temp,'r')
            result.append(t)
        #Populating the bagging dataset with predictions for each bag    
        bagging[elem] = result
    
    bagging['majority']=''
    #Finding majority for each bag across one particular observation
    for j in range(0,len(bagging)):
       
        bagging['majority'].iloc[j] = mode(list(bagging.iloc[j])[0:(x+1)])
    
    #Performnace measures
    x = accuracy_score(test_set['survived'],bagging['majority'])*100
    y = precision_score(test_set['survived'],bagging['majority'])
    z = recall_score(test_set['survived'],bagging['majority'])
    
    #Append accuracy score, precision, recall for each value of B from bags
    acc.append(x)
    print('Accuracy',x)
    p.append(y)
    print('Precision',y)
    r.append(z)
    print('Recall',z)
    

Bags: 6


C:\Users\Neha\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


  node observation   Survived    Drowned feature  value
1    r        1046  37.380497  62.619503     sex      0
2   rL         366  75.136612  24.863388  pclass      2
3  rLL         166  54.819277  45.180723    fare  24.15
  node observation   Survived    Drowned feature value
1    r        1046  38.240918  61.759082     sex     0
2   rL         402  71.144279  28.855721  pclass     2
3  rLL         189  45.502646  54.497354     age     1
  node observation   Survived    Drowned feature  value
1    r        1046  34.799235  65.200765     sex      0
2   rL         362  71.270718  28.729282  pclass      2
3  rLL         183  48.633880  51.366120    fare  23.25
  node observation   Survived    Drowned feature    value
1    r        1046  36.902486  63.097514     sex        0
2   rL         364  71.428571  28.571429  pclass        2
3  rLL         172  47.093023  52.906977    fare  31.3875
  node observation   Survived    Drowned feature  value
1    r        1046  38.240918  61.759082    

C:\Users\Neha\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Accuracy 77.86259541984732
Precision 0.8409090909090909
Recall 0.6271186440677966
Bags: 10
  node observation   Survived    Drowned feature  value
1    r        1046  36.137667  63.862333     sex      0
2   rL         379  70.976253  29.023747  pclass      2
3  rLL         181  46.961326  53.038674    fare  23.25
  node observation   Survived    Drowned   feature value
1    r        1046  36.424474  63.575526       sex     0
2   rL         374  68.716578  31.283422    pclass     2
3  rLL         168  38.690476  61.309524  embarked     1
  node observation   Survived    Drowned feature    value
1    r        1046  39.101338  60.898662     sex        0
2   rL         408  69.607843  30.392157  pclass        2
3  rLL         214  47.196262  52.803738    fare  31.3875
  node observation   Survived    Drowned feature  value
1    r        1046  37.571702  62.428298     sex      0
2   rL         373  69.973190  30.026810  pclass      2
3  rLL         176  44.318182  55.681818    fare  24.15
 

In [44]:
acc = [value/100 for value in acc]
print(acc)
print(p)

print(r)

[0.7786259541984732, 0.7595419847328244]
[0.8409090909090909, 0.8666666666666667]
[0.6271186440677966, 0.5508474576271186]


Random Forest

In [46]:
training_set = pd.concat([X_train,y_train],axis=1)
test_set = pd.concat([X_test,y_test],axis=1)

In [47]:
forest = [5,9]
acc=[]
p=[]
r=[]
for x in forest:
    print('Forest with trees =',x+1)
    cols = []
    for i in (range(0,x)):
        a = 'bag'+str(i)
        cols.append(a)

    bagging = pd.DataFrame(columns = cols)

    for elem in bagging.columns:
        
        #Defining a tree to store decision tree results for each individual bag
        result_tree = pd.DataFrame(columns = ['node', 'observation', 'Survived', 'Drowned', 'feature', 'value'])
       
        cols1 = training_set.columns
        cols1 = cols1[cols1 != 'survived']
        
        #Selecting random number of features m = sqrt.(p)
        list_cols = (np.random.choice(cols1, size=int(round(math.sqrt(len(cols1)),0)), replace = False))
        list_cols = list(np.append(list_cols, 'survived'))
        
        #Subsetting the dataset
        set_train = training_set[list_cols]
               
        
        #Drawing N samples with replacement from N observations
        bag_train = set_train.sample(n=len(set_train), replace = True)
        
        #Running decision tree to populate result_tree dataframe
        decision_tree(bag_train, 'r')
        
        print(result_tree.head(3))

        result = []

        d_final_test = test_set[['pclass', 'age', 'sibsp', 'parch', 'fare', 'sex', 'embarked']].dropna()

        #Making predictions on the test set
        for e in d_final_test.iterrows():
            temp = pd.DataFrame(list(e)[1:], columns = ['pclass', 'age', 'sibsp', 'parch', 'fare', 'sex', 'embarked'])
            t = prediction(temp,'r')
            result.append(t)
        #Populating the bagging dataset with predictions for each bag    
        bagging[elem] = result
    
    bagging['majority']=''
    #Finding majority for each bag across one particular observation
    for j in range(0,len(bagging)):
        bagging['majority'].iloc[j] = mode(list(bagging.iloc[j])[0:(x+1)])
    
    #Performnace measures
    x = accuracy_score(test_set['survived'],bagging['majority'])
    y = precision_score(test_set['survived'],bagging['majority'])
    z = recall_score(test_set['survived'],bagging['majority'])
    
    #Append accuracy score, precision, recall for each value of B from bags
    acc.append(x)
    print('Accuracy',x)
    p.append(y)
    print('Precision',y)
    r.append(z)
    print('Recall',z)
    

Forest with trees = 6


C:\Users\Neha\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


  node observation   Survived    Drowned   feature    value
1    r        1047  36.771729  63.228271      fare  15.0458
2   rL         497  51.106640  48.893360  embarked   0.0000
3  rLL         364  43.131868  56.868132      fare  77.2875
  node observation   Survived    Drowned feature    value
1    r        1047  36.962751  63.037249    fare  15.1000
2   rL         495  51.111111  48.888889    fare  75.2417
3  rLL         102  75.490196  24.509804   parch   0.0000
  node observation   Survived    Drowned   feature value
1    r        1047  36.580707  63.419293    pclass     2
2   rL         528  25.189394  74.810606  embarked     1
3  rLL         361  18.559557  81.440443     sibsp     4
  node observation   Survived    Drowned feature value
1    r        1047  39.063992  60.936008     sex     0
2   rL         389  74.807198  25.192802  pclass     2
3  rLL         171  49.707602  50.292398   sibsp     2
  node observation   Survived    Drowned feature  value
1    r        1047  34.6

C:\Users\Neha\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Accuracy 0.6755725190839694
Precision 0.7391304347826086
Recall 0.4322033898305085
Forest with trees = 10
  node observation   Survived    Drowned feature  value
1    r        1047  37.058262  62.941738    fare   15.1
2   rL         493  51.926978  48.073022  pclass    2.0
3  rLL         142  30.985915  69.014085    fare   17.4
  node observation   Survived    Drowned   feature value
1    r        1047  35.721108  64.278892  embarked     0
2   rL         848  31.603774  68.396226     parch     0
3  rLL         186  45.161290  54.838710     sibsp     2
  node observation   Survived    Drowned feature value
1    r        1047  37.535817  62.464183  pclass     1
2   rL         791  29.582807  70.417193   parch     0
3  rLL         167  46.706587  53.293413  pclass     2
  node observation   Survived    Drowned feature value
1    r        1047  38.108883  61.891117     sex     0
2   rL         397  71.284635  28.715365    fare  46.9
3  rLL         124  92.741935   7.258065   parch     1
  

There is no significant increase in the accuracy or precision. Due to limitations in performing calculations, testing for high values like B=1000 could not be done